In [34]:
import torch
from torchvision import datasets, transforms
from torch.utils.data import random_split, DataLoader
import torch.nn as nn
import torch.nn.functional as F
import numpy
from torch.optim.lr_scheduler import ExponentialLR

In [ ]:
transform = transforms.Compose([
    transforms.ToTensor(),
])

In [18]:
device = "cuda"

In [14]:
dataset = datasets.ImageFolder(root='dataset', transform=transform)

In [15]:
train_size = int(0.8 * len(dataset))
test_size = int((len(dataset) - train_size)/2)
valid_size = test_size+1

train_ds, test_ds, validate_ds = random_split(dataset, [train_size, test_size, valid_size])

In [16]:
train_loader = DataLoader(train_ds, batch_size=32, shuffle=True)
test_loader = DataLoader(test_ds, batch_size=32, shuffle=True)
valid_loader = DataLoader(validate_ds, batch_size=32, shuffle=True)

In [43]:
class BlightModel(nn.Module):
    def __init__(self):
        super(BlightModel, self).__init__()

        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)
        self.bn1 = nn.BatchNorm2d(32)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.bn2 = nn.BatchNorm2d(64)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.bn3 = nn.BatchNorm2d(128)
        self.conv4 = nn.Conv2d(128, 256, kernel_size=3, padding=1)
        self.bn4 = nn.BatchNorm2d(256)

        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)

        self.fc1 = nn.Linear(256*16*16, 256)
        self.fc2 = nn.Linear(256, 3) 

    def forward(self, x):
        x = self.pool(F.relu(self.bn1(self.conv1(x))))
        x = self.pool(F.relu(self.bn2(self.conv2(x))))
        x = self.pool(F.relu(self.bn3(self.conv3(x))))
        x = self.pool(F.relu(self.bn4(self.conv4(x))))

        x = x.view(x.size(0), -1)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x    

In [44]:
model = BlightModel()
model.to(device)

BlightModel(
  (conv1): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv3): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn3): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv4): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn4): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=65536, out_features=256, bias=True)
  (fc2): Linear(in_features=256, out_features=3, bias=True)
)

In [45]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(params=model.parameters(), lr=0.01)
scheduler = ExponentialLR(optimizer, gamma=0.95)

In [ ]:
epochs = 10

for epoch in range(epochs):
    model.train()
    total_loss = 0
    correct = 0
    total = 0

    for input, label in train_loader:
        input = input.to(device)
        label = label.to(device)

        optimizer.zero_grad()
        output = model(input)

        loss = loss_fn(output, label)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

        _, pred = torch.max(output, 1)
        total += label.size(0)
        correct += (pred==label).sum().item()

    train_loss = total_loss/len(train_loader)
    train_accuracy = 100*correct/total

    val_loss = 0
    val_correct = 0
    val_total = 0

    with torch.no_grad():
        for input, label in valid_loader:
            input = input.to(device)
            label = label.to(device)

            output = model(input)
            loss = loss_fn(output, label)
            val_loss += loss.item()

            _, pred = torch.max(output, 1)
            val_total += label.size(0)
            val_correct += (pred==label).sum().item()

    val_loss = val_loss/len(valid_loader)
    val_accuracy = 100*val_correct/val_total

    print(f"Epoch:{epoch} Train Loss:{train_loss}  Train Acc:{train_accuracy}  Val loss:{val_loss}  Val Accuracy:{val_accuracy}")

Epoch:0 Train Loss:37.44762754799039  Train Acc:77.91981406159209  Val loss:2.016429271016802  Val Accuracy:72.22222222222223
Epoch:1 Train Loss:0.42545686686342515  Train Acc:90.70307960488088  Val loss:0.2273379479135786  Val Accuracy:92.12962962962963
Epoch:2 Train Loss:0.14074075236675088  Train Acc:94.59616502033701  Val loss:0.13962397298642568  Val Accuracy:94.9074074074074
Epoch:3 Train Loss:0.09779916010383102  Train Acc:96.22312608948286  Val loss:0.14157138179455483  Val Accuracy:93.98148148148148
Epoch:4 Train Loss:0.10564123235504937  Train Acc:96.57176060429983  Val loss:0.14750679316265242  Val Accuracy:94.9074074074074
Epoch:5 Train Loss:0.09365116525441408  Train Acc:96.51365485183032  Val loss:0.11037390120327473  Val Accuracy:95.37037037037037
Epoch:6 Train Loss:0.06789163637613119  Train Acc:97.03660662405578  Val loss:0.07963257655501366  Val Accuracy:94.9074074074074
Epoch:7 Train Loss:0.06981348004828086  Train Acc:97.55955839628123  Val loss:0.14817932061851025 

In [48]:
with torch.inference_mode():
    test_loss = 0
    test_correct = 0
    test_total = 0

    for input, label in test_loader:
        input = input.to(device)
        label = label.to(device)

        output = model(input)
        loss = loss_fn(output, label)
        test_loss += loss.item()

        _, pred = torch.max(output, 1)
        test_total += label.size(0)
        test_correct += (pred==label).sum().item()

    test_loss = test_loss/len(valid_loader)
    test_accuracy = 100*test_correct/test_total

    print(f"Test Loss: {test_loss}  Test Accuracy: {test_accuracy}")

Test Loss: 0.19052207150629588  Test Accuracy: 93.95348837209302


In [49]:
torch.save(model.state_dict(), "Blight_model.pth")